In [ ]:
import requests             
from bs4 import BeautifulSoup
import csv                  
import webbrowser
import re
import csv
import numpy as np

from time import sleep
from random import randint
 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import exceptions 

In [ ]:
url = "https://www.tripadvisor.co.uk/Hotel_Review-g186338-d187591-Reviews-or5-The_Ritz_London-London_England"

try:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser') #prints out the pagesource
except:
    print('An error occured') #incase an error occurs

In [ ]:
# Prepare CSV
csv_file = open ('tripadvisor_scrape.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['profile name', 'profile contributions', 'profile helpful votes', 'review title', 'rating', 'review', 'date of stay', 'trip category'])

In [ ]:
%%time 

chrome_path = r"/Users/louisemurray-bruce/chromedriver"
driver = webdriver.Chrome(chrome_path)

#Initialize the number of pages 
pages = range(0, 4411, 5)

for page in pages:
    url = "https://www.tripadvisor.co.uk/Hotel_Review-g186338-d187591-Reviews-" + "or"+str(page) + "-The_Ritz_London-London_England"
    driver.get(url) 

    try:
        buttons = WebDriverWait(driver, 5).until(
        EC.visibility_of_all_elements_located((By.XPATH, "//span[@class='_3maEfNCR']")))
    except:
        pass
    else:
        for i in range(len(buttons)):
            driver.execute_script("document.getElementsByClassName('_3maEfNCR')[arguments[0]].click()", i)

    #using beautiful        
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Pause the loop
    sleep(randint(2,10))

    for review_container in soup.find_all('div', class_= '_2wrUUKlw _3hFEdNs8'):
        # extract review title
        review_title = review_container.find('div', class_= 'glasR4aX')
        if review_title:
            review_title = review_title.text
        else:
            review_title = 'nan'

        # extract number of contributions and number of helpful votes
        items = review_container.find_all('span', class_ ='_3fPsSAYi')
        if len(items) > 0:
            contributions = items[0].text
            contributions = contributions.split()[0]
        else:
            contributions= '0'
        if len(items) > 1:
            helpful_votes = items[1].text
            helpful_votes = helpful_votes.split()[0]
        else:
            helpful_votes = '0'

        # extract reviewers date of stay
        date = review_container.find('span', class_ = '_34Xs-BQm')
        if date:
            date= date.text#Split on the column to extract the date of stay
            date_of_stay = date.split(':')[1]
        else:
            date_of_stay = 'nan'

        # extract review
        review = review_container.find('div', class_ ='cPQsENeY').text

        # extract name
        name = review_container.find('a', class_='ui_header_link _1r_My98y')
        if name:
            name = name.text
        else:
            name = 'nan'

        # extract trip type
        trip_category = review_container.find('span', class_ = '_2bVY3aT5')
        if trip_category:
            trip_category= trip_category.text.split(':')[1]
        else:
            trip_category = 'nan'

        #bubble review rating 
        bubblerating = review_container.find('div', class_ = 'nf9vGX55')
        if bubblerating:
            ratings = re.findall('\d+', ''.join(map(str, bubblerating)))
            #convert to integers and divide by 10 to get the true rating
            ratings = [int(i) for i in ratings]
            my_int = 10
            new_rating= [i/my_int for i in ratings]
        else:
            new_rating = 'nan'

        csv_writer.writerow([name, contributions, helpful_votes, review_title, new_rating, review, date_of_stay, trip_category])

csv_file.close()